In [ ]:
import os
from pathlib import Path

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = str(Path.cwd().joinpath("cache"))

# Test Langevin Dynamics by Gaussian Model

In [ ]:
import matplotlib.pyplot as plt
import torch

from gaussian import GaussianModel
from gmm import GaussianMixtureModel
from langevin import (
    LangevinGaussianModelPipeline,
    LangevinNoiseScheduler
)

In [ ]:
def draw_inference_result(
    pipeline: LangevinNoiseScheduler,
    batch_size: int,
    num_inference_steps: int,
    seed: int
):
    plt.figure(figsize=(10, 10))

    algorithm_types = ["ode", "sde"]
    random_samplings = [True, False]

    for row_id, algorithm_type in enumerate(algorithm_types):
        pipeline.scheduler.config.algorithm_type = algorithm_type
        for col_id, random_sampling in enumerate(random_samplings):
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)

            samples = pipeline(batch_size=batch_size, num_inference_steps=num_inference_steps, random_sampling=random_sampling)
            samples = samples.detach().cpu().numpy()
            mean, std = samples.mean(axis=0), samples.std(axis=0)
            print(f"{algorithm_type} {random_sampling}: mean={mean}, std={std}")

            plt.subplot(len(algorithm_types), len(random_samplings), row_id * len(random_samplings) + col_id + 1)
            plt.hist2d(samples[:, 0], samples[:, 1], bins=[128, 128], range=[[-4, 4], [-4, 4]], cmap="plasma", cmin=1)
            plt.title(f"{algorithm_type} {random_sampling}")
            plt.axis("equal")

    plt.show()

In [ ]:
seed = 0
batch_size = 16384
num_inference_steps = 1000

In [ ]:
pipeline = LangevinGaussianModelPipeline(
    GaussianModel(
        mu=torch.tensor([0.0, 0.0]),
        cov=torch.tensor([[1.0, 0.5], [0.5, 1.0]])
    ),
    LangevinNoiseScheduler(
        t_min=0.0,
        t_max=1.0,
        algorithm_type="sde",
    )
)
if torch.cuda.is_available():
    pipeline = pipeline.to(device=torch.device("cuda"))

In [ ]:
draw_inference_result(pipeline, batch_size, num_inference_steps, seed)

In [ ]:
pipeline = LangevinGaussianModelPipeline(
    GaussianMixtureModel(
        num_groups_per_model=8,
        num_gs_per_group=8,
        model_radius=2,
        group_radius=0.4,
        sigma=0.04
    ),
    LangevinNoiseScheduler(
        t_min=0.0,
        t_max=1.0,
        algorithm_type="sde",
    )
)
if torch.cuda.is_available():
    pipeline = pipeline.to(device=torch.device("cuda"))

In [ ]:
draw_inference_result(pipeline, batch_size, num_inference_steps, seed)